In [1]:
import pandas as pd
from tools import *
from CONSTANT import *
import os

from tqdm import tqdm

import matplotlib.pyplot as plt

%matplotlib inline

# HKU956

In [2]:
av_rating = pd.read_csv(r'./HKU956/3. AV_ratings_duration.csv')
av_rating['valence'] = av_rating['valence'].apply(lambda x: 1 if x == 'positive' else 0)
av_rating['arousal'] = av_rating['arousal'].apply(lambda x: 1 if x == 'positive' else 0)
av_rating.head()

,participant_id,song_no,song_id,valence_rating,valence,arousal_rating,arousal,play_duration
0,hku1919,16,370177,9.3,1,1.0,1,72
1,hku1919,8,1119687,9.8,1,-3.0,0,73
2,hku1919,18,77933,-0.1,0,1.9,1,41
3,hku1919,14,238585,1,1,-1.8,0,66
4,hku1919,20,1168711,0.2,1,-3.4,0,33


In [8]:
def signal_processer(user, signal_name, signal_files, win_size=4, step=2):
    all_signals = {}
    tmp_signals = []
    va_info_col = ['participant_id', 'song_no', 'song_id', 'valence_rating',
                   'valence', 'arousal_rating', 'arousal', 'play_duration']
    for file in signal_files:
        try:
            filename = os.path.split(file)[-1]
            filename, file_extension = os.path.splitext(filename)
            if file_extension != '.csv':
                continue
            # songno, songid = filename.split('_')
            
            signal = pd.read_csv(file, header=None).iloc[:,0].values

            # bandpass or lowpass filtering & resampling
            if signal_name == 'EDA':
                signal = butter_lowpass_filter(signal, cutOff=CUTOFF['EDA'], fs=SAMPLERATE['EDA'])
                signal = resample_by_interpolation(signal, input_fs=SAMPLERATE['EDA'], output_fs=100)
            elif signal_name in ['TEMP', 'BVP']:
                signal = butter_bandpass_filter(signal, lowcut=CUTOFF[signal_name][0], highcut=CUTOFF[signal_name][1], fs=SAMPLERATE[signal_name])
                signal = resample_by_interpolation(signal, input_fs=SAMPLERATE[signal_name], output_fs=100)
            elif signal_name == 'HR':
                signal = chauvenet_filter(signal)
                # print(signal)
                signal = resample_by_interpolation(signal, input_fs=SAMPLERATE[signal_name], output_fs=100)
            else: # IBI
                continue

            tmp_signals.extend(signal.tolist())
            all_signals[filename] = signal
            
        except Exception as e:
            print(e, file)
            raise e
        
    signal_max = np.max(tmp_signals)
    signal_min = np.min(tmp_signals)

    del tmp_signals

    signal_df = []

    # normalization, segmentation, $ concatenation
    for filename, signal in all_signals.items():
        song_no, song_id = filename.split('_')
        signal = (signal - signal_min) / (signal_max - signal_min)
        signal = pd.Series(signal).interpolate().tolist()
        
        segments = np.array(segment_generator(signal, win_size=win_size*100, step=step*100)[1:])
        seg_cols = ['{}_seg{}'.format(signal_name, i) for i in range(segments.shape[1])]
        segment_df = pd.DataFrame(columns=seg_cols, data=segments)

        segment_df['segment_id'] = segment_df.index.tolist()
        segment_df['song_no'] = int(song_no)
        segment_df['song_id'] = int(song_id)
        segment_df['user'] = user

        # va_info = [av_rating[(av_rating['participant_id']==user) & (av_rating['song_no']==int(songno)) & (av_rating['song_id']==int(songid))].values.tolist()[0] for _ in range(len(segments))]
        # tmp_va_info = pd.DataFrame(columns=va_info_col, data=va_info)
        # segment_df = pd.concat([tmp_va_info, segment_df], axis=1)
        
        signal_df.append(segment_df)
    
    return pd.concat(signal_df)

In [10]:
users = os.listdir(HKU_DIR)
# for signal_name in ['HR']:
for signal_name in SIGNALS:
    if signal_name == 'IBI':
        continue
    signals = []
    for user in users:
        signal_files = get_folder_files(os.path.join(HKU_DIR, user, signal_name))
        signal_df = signal_processer(user, signal_name, signal_files)
        signals.append(signal_df)
        # break
    signals = pd.concat(signals)
    signals.to_csv(os.path.join(PROCESSED_DIR, 'HKU956', '{}.csv'.format(signal_name)), index=False)
    print(signal_name, signals.shape)

HR (22727, 404)


In [11]:
hr = pd.read_csv(r'./HKU956/1. physiological_signals/hku1903/HR/0_262957.csv', header=None).iloc[:,0].values

In [41]:
np.ma.masked_array(data=hr, mask=criterion(hr), fill_value=np.nan).filled()

array([71.03, 70.97, 70.97, 70.95, 70.95, 71.  , 71.05, 71.12, 71.18,
       71.15, 71.12, 71.08, 71.05, 71.03, 71.03, 71.03, 71.03, 71.02,
       71.05, 71.08, 71.12, 71.18, 71.2 , 71.12, 71.08, 71.03, 71.02,
       71.  , 71.03, 71.07, 71.12, 71.18, 71.3 , 71.42, 71.57, 71.73,
       71.9 , 72.08, 72.25, 72.38, 72.48, 72.63, 72.8 , 72.95, 73.1 ,
       73.23, 73.35, 73.47, 73.45, 73.42, 73.4 , 73.38, 73.35, 73.33,
       73.32, 73.3 , 73.28, 73.28, 73.28, 73.3 , 73.32, 73.37, 73.38,
       73.38, 73.38, 73.35, 73.28, 73.23, 73.22, 73.32, 73.42, 73.53,
       73.65, 73.75, 73.82, 73.87, 73.88, 73.88, 73.87, 73.83, 73.78,
       73.72, 73.67, 73.63, 73.6 , 73.58, 73.57, 73.57, 73.53, 73.5 ,
       73.45, 73.38, 73.33, 73.27, 73.2 , 73.15, 73.12, 73.12, 73.12,
       73.15, 73.22, 73.27, 73.32, 73.4 , 73.5 , 73.6 , 73.7 , 73.82,
       73.9 , 74.13, 74.35, 74.53, 74.73, 74.95, 75.23, 75.55, 75.85,
       76.13, 76.33, 76.52, 76.7 , 76.83, 77.02, 77.23, 77.43, 77.62,
       77.8 , 77.97,

In [24]:
a = np.array([1, 2, 3]).astype(int)
mask = np.array([True, False, True])

In [16]:
a[mask]

array([1, 3])

In [35]:
M = np.ma.masked_array(data=np.array([1.0, 2.0, 3.0]), mask=mask, fill_value=np.nan).filled()

In [39]:
pd.Series(M).interpolate()

0    NaN
1    2.0
2    2.0
dtype: float64